In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech
import math
import gc  # for garbage collection

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\cnn8grps_rad1_model.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        cv2.circle(white, (x - x_min + os_x, y - y_min + os_y), 5, (0, 0, 255), -1)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    white_resized = cv2.resize(white, (400, 400))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    self.predict(white_expanded, hand_landmarks)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

            # Force garbage collection
            gc.collect()

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def predict(self, test_image, hand_landmarks):
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        self.custom_prediction_logic(ch1, ch2, ch3, hand_landmarks)

    def custom_prediction_logic(self, ch1, ch2, ch3, hand_landmarks):
        def distance(pt1, pt2):
            return math.sqrt((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2)

        landmarks = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

        distances = {
            'index_middle': distance(landmarks[8], landmarks[12]),
            'index_ring': distance(landmarks[8], landmarks[16]),
            'index_pinky': distance(landmarks[8], landmarks[20]),
            'thumb_index': distance(landmarks[4], landmarks[8]),
            'thumb_middle': distance(landmarks[4], landmarks[12]),
            'thumb_ring': distance(landmarks[4], landmarks[16]),
            'thumb_pinky': distance(landmarks[4], landmarks[20]),
            'thumb_tip_palm': distance(landmarks[4], landmarks[0]),
            'index_palm': distance(landmarks[8], landmarks[0]),
            'middle_palm': distance(landmarks[12], landmarks[0]),
            'ring_palm': distance(landmarks[16], landmarks[0]),
            'pinky_palm': distance(landmarks[20], landmarks[0]),
            'thumb_knuckle_palm': distance(landmarks[3], landmarks[0])
        }

        self.current_symbol = ascii_uppercase[ch1]

        if self.current_symbol == 'B' and distances['index_middle'] < distances['index_palm']:
            self.current_symbol = 'A'
        elif self.current_symbol == 'C' and distances['thumb_pinky'] < distances['pinky_palm']:
            self.current_symbol = 'O'
        elif self.current_symbol == 'D' and distances['thumb_index'] > distances['thumb_tip_palm']:
            self.current_symbol = 'L'
        elif self.current_symbol == 'Y' and distances['thumb_pinky'] > distances['pinky_palm']:
            self.current_symbol = 'Y'

        self.update_sentence()

    def update_sentence(self):
        self.ct[self.current_symbol] += 1

        if self.ct[self.current_symbol] > 20:
            self.ct = {char: 0 for char in ascii_uppercase}
            if self.current_symbol != self.prev_char:
                self.str += self.current_symbol + " "
                self.prev_char = self.current_symbol

        self.panel3.config(text=self.current_symbol, font=("Courier", 30))
        self.panel5.config(text=self.str, font=("Courier", 30))

    def action1(self):
        idx_space = self.str.rfind(" ")
        idx_word = self.str.find(self.word, idx_space)
        last_idx = len(self.str)
        self.str = self.str[:idx_word]
        self.str = self.str + self.word1.upper()


    def action2(self):
        idx_space = self.str.rfind(" ")
        idx_word = self.str.find(self.word, idx_space)
        last_idx = len(self.str)
        self.str=self.str[:idx_word]
        self.str=self.str+self.word2.upper()
        #self.str[idx_word:last_idx] = self.word2


    def action3(self):
        idx_space = self.str.rfind(" ")
        idx_word = self.str.find(self.word, idx_space)
        last_idx = len(self.str)
        self.str = self.str[:idx_word]
        self.str = self.str + self.word3.upper()



    def action4(self):
        idx_space = self.str.rfind(" ")
        idx_word = self.str.find(self.word, idx_space)
        last_idx = len(self.str)
        self.str = self.str[:idx_word]
        self.str = self.str + self.word4.upper()
    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30))

if __name__ == "__main__":
    Application()
    tk.mainloop()


Loading model...
Model loaded successfully.
1/1 [==============================] - 0s 47ms/step
Closing Application
